In [1]:
import os
import wfdb
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import resample
from data_storage_utils import normalizer
from parameters_CINC11CINC17_balanced import parameters
parameters = parameters()

data_length = parameters['data_length']
wav_data_range = parameters['wav_data_range']
input_data_folder = parameters['path_BUTQDB']
output_folder = parameters['path_BUTQDB_test']

In [4]:
ecg_numbers = os.listdir(input_data_folder)
for number in ecg_numbers:
    with open(f'{input_data_folder}//{number}//{number}_ANN.csv') as f:
        lines = f.readlines()

        for line in lines:
            line = line.split(',')
            
            if (int(line[-2]) - int(line[-3]) >= data_length*2):
                label = 1 # everything except label 3 is set to 1 ( acceptable )
                if line[-1][0] == '3':
                    label = 0 # set to 0 ( unacceptable ) if label 3
                    
                segment_length = int(line[-2]) - int(line[-3])
                # get ecg recording from start label to end label
                record = wfdb.rdrecord(f'{input_data_folder}//{number}//{number}_ECG', sampfrom=int(line[-3]), sampto=int(line[-2])) 
                start_data = 0 # start at index 0 in recording
                for end_data in range(data_length*2, segment_length, data_length*2): # iterate over segments in recoding
                    data = record.__dict__['p_signal'][start_data:end_data]
                    data = resample(data, data_length) # convert 1000Hz to 500Hz
                    data = normalizer(data, wav_data_range)
                    data = np.asarray(data, dtype=np.int16)
                    wavfile.write(f'{output_folder}\\{label}\\{number}_{line[-3]}_{start_data}.wav', 500, data)
                    
#                     file = wavfile.read(f'{output_folder}\\{label}\\{number}_{line[-3]}_{start_data}.wav')
#                     plt.plot(data)
#                     plt.title(f'{number}_{line[-3]}_{int(line[-3])+start_data} {int(line[-3])+end_data}')
#                     plt.show()

                    start_data = end_data # start next data segement at end of last